In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install -U ultralytics optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 28.9 MB/s eta 0:00:00


In [3]:
from ultralytics import YOLO
import torch
import os
import optuna
import json

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Rutas
DATASET_ROOT = "/content/drive/MyDrive/Prueba"

DATA_YAML = os.path.join(DATASET_ROOT, "data.yaml")

YOLO_ROOT = "/content/drive/MyDrive/Prueba/Yolo_Experiments"
os.makedirs(YOLO_ROOT, exist_ok=True)

NUM_CLASSES = 4   # door, window, balcony, railing

In [5]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", DEVICE)


Device: cpu


In [6]:
def objective(trial):
    # Hyperparametros para optuna
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW"])

    # Modelo
    model = YOLO("yolov8l.pt")

    model.train(
        data=DATA_YAML,
        epochs=15,
        patience=3,  # para si no mejora mAP durante 3 épocas consecutivas
        imgsz=512,
        batch=-1,  # YOLOv8 calcula AUTOMÁTICAMENTE el máximo batch_size que cabe en tu GPU
        lr0=lr,
        optimizer=optimizer_name,
        freeze=10,      # congelar primeras capas (backbone)
        device=DEVICE,  # cuda
        project=YOLO_ROOT,
        name=f"trial_{trial.number}",
        save=True,      # guardar best.pt / last.pt
        exist_ok=True,  # no sobrescribir conflictos
        cache=False,
        verbose=True
    )

    # Evaluación final
    val_results = model.val()

    return 1 - float(val_results.box.map50)


In [7]:
# Crear estudio
study = optuna.create_study(direction="minimize")

# Ejecutar Optuna
study.optimize(objective, n_trials=10)

print("Mejor configuración encontrada:")
print(study.best_params)


[I 2026-01-23 20:45:38,623] A new study created in memory with name: no-name-54212398-9d21-4b3b-a136-526476d7979d


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Prueba/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0011575541660221427, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_0, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/conte

[I 2026-01-23 20:50:21,264] Trial 0 finished with value: 0.7382118781214521 and parameters: {'lr': 0.0011575541660221427, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.7382118781214521.


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Prueba/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.006963556241741474, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_1, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/conten

[I 2026-01-23 20:54:16,528] Trial 1 finished with value: 0.638408413069708 and parameters: {'lr': 0.006963556241741474, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.638408413069708.


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Prueba/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.004556919670974512, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_2, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/conten

[I 2026-01-23 20:58:22,071] Trial 2 finished with value: 0.6666239530911697 and parameters: {'lr': 0.004556919670974512, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.638408413069708.


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Prueba/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005418144221077154, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_3, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/conten

[I 2026-01-23 21:02:29,473] Trial 3 finished with value: 0.6318753084472397 and parameters: {'lr': 0.005418144221077154, 'optimizer': 'Adam'}. Best is trial 3 with value: 0.6318753084472397.


Mejor configuración encontrada:
{'lr': 0.005418144221077154, 'optimizer': 'Adam'}


In [17]:
# Cargar el mejor trial
best_trial_number = study.best_trial.number
best_trial_dir = os.path.join(YOLO_ROOT, f"trial_{best_trial_number}")
best_model_path = os.path.join(best_trial_dir, "weights", "best.pt")

# Cargar modelo YOLO
from ultralytics import YOLO
best_model = YOLO(best_model_path)

# Evaluar validación final
results = best_model.val()
print(results.box.map50)


Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
Model summary (fused): 113 layers, 43,609,692 parameters, 0 gradients, 164.8 GFLOPs
val: Fast image access ✅ (ping: 0.7±0.4 ms, read: 8.6±3.1 MB/s, size: 23.8 KB)
val: Scanning /content/drive/MyDrive/Prueba/val/labels.cache... 5 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 5/5 1.1Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 11.2s/it 11.2s
                   all          5         53      0.588      0.479      0.368      0.217
                  door          5          6          1          0     0.0206      0.014
                window          5         47      0.176      0.957      0.716       0.42
Speed: 1.2ms preprocess, 2148.1ms inference, 0.0ms loss, 77.1ms postprocess per image
Results saved to /content/runs/detect/val7
0.3681246915527603
